# CSE 572: Data Mining 
## Homework 3 Task 1
## Varad Vijay Deshmukh
## 1225369184

### Task 1 Algorithmic Analysis K-Means Clustering with Real World Dataset

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/k-means-data/data_description.txt
/kaggle/input/k-means-data/label.csv
/kaggle/input/k-means-data/data.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import pairwise_distances

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
data = np.array(pd.read_csv('/kaggle/input/k-means-data/data.csv', header=None))

In [5]:
labels = np.ravel(pd.read_csv('/kaggle/input/k-means-data/label.csv', header=None))

In [6]:
print(data.shape)
print(labels.shape)

(10000, 784)
(10000,)


**Q1: Run K-means clustering with Euclidean, Cosine and Jarcard similarity. Specify K= the**
**number of categorical values of y (the number of classifications). Compare the SSEs of** 
**Euclidean-K-means, Cosine-K-means, Jarcard-K-means. Which method is better?**

In [7]:
class KMeansClustering:
    
    def __init__(self, k, stopping_criterion="no_change") -> None:
        self.k = k
        self.stopping_criterion = stopping_criterion
        self.centroids = None
        self._sse_score = None
        self._last_sse_score = float('inf')
        self._iterations = 0

    def euclidean_distance(self, data_point, centroids):
        return np.sqrt(np.sum((centroids - data_point)**2, axis=1))

    def __sum_of_squared_errors_calc(self, centroids, data, y):
        sum_of_errors = 0.0
        for idx, d in enumerate(data):
            sum_of_errors += np.sum((centroids[y[idx]] - d) ** 2)

        return sum_of_errors

    def get_sum_of_squared_error(self):
        return self._sse_score
    def get_iterations_to_converge(self):
        return self._iterations
    
    def fit(self, X, max_iterations=200):
        self.centroids = np.random.uniform(
            low=np.amin(X, axis=0),
            high=np.amax(X, axis=0),
            size=(self.k, X.shape[1]))
        y = []
        for _ in range(max_iterations):
            y = []
            for data_point in X:
                distances = self.euclidean_distance(data_point=data_point,centroids=self.centroids)
                cluster_num = np.argmin(distances)
                y.append(cluster_num)
            y = np.asarray(y)
            cluster_indices = []
            for idx in range(self.k):
                cluster_indices.append(np.argwhere(y == idx))
            cluster_centers = []
            for i, indices in enumerate(cluster_indices):
                if len(indices) == 0:
                    cluster_centers.append(self.centroids[i])
                else:
                    cluster_centers.append(np.mean(X[indices], axis=0)[0])
            if self.stopping_criterion == "no_change" and np.max(self.centroids - np.array(cluster_centers)) < 1e-3:
                break
            elif self.stopping_criterion == "increase_sse":
                current_sse = self.__sum_of_squared_errors_calc(X, np.array(cluster_centers), y)
                if current_sse > self._last_sse_score:
                    break
                self._last_sse_score = current_sse
            else:
                self.centroids = np.array(cluster_centers)
            self._iterations += 1
        self._sse_score = self.__sum_of_squared_errors_calc(X, self.centroids, y)
        return y   


In [7]:
class KMeansClustering:
    
    def __init__(self, k, stopping_criterion="no_change") -> None:
        self.k = k
        self.stopping_criterion = stopping_criterion
        self.centroids = None
        self._sse_score = None
        self._last_sse_score = float('inf')
        self._iterations = 0

    def euclidean_distance(data_point, centroids):
        return np.sqrt(np.sum((centroids - data_point) ** 2, axis=1))

    def _sum_of_squared_errors(self, data, assignments):
        return sum(np.sum((self.centroids[cluster] - data[point_idx]) ** 2)
                   for point_idx, cluster in enumerate(assignments))

    def get_sum_of_squared_error(self):
        return self._sse_score

    def get_iterations_to_converge(self):
        return self._iterations
    
    def fit(self, X, max_iterations=200):
        self.centroids = np.random.uniform(low=np.amin(X, axis=0), high=np.amax(X, axis=0), size=(self.k, X.shape[1]))

        for _ in range(max_iterations):
            distances = np.array([self.euclidean_distance(data_point, self.centroids) for data_point in X])
            assignments = np.argmin(distances, axis=1)
            new_centroids = np.array([X[assignments == k].mean(axis=0) if len(X[assignments == k]) > 0 else self.centroids[k] 
                                      for k in range(self.k)])
            if self.stopping_criterion == "no_change" and np.allclose(self.centroids, new_centroids, atol=1e-3):
                #print("breaking -check")
                break
            elif self.stopping_criterion == "increase_sse":
                current_sse = self._sum_of_squared_errors(X, assignments)
                if current_sse > self._last_sse_score:
                    break
                self._last_sse_score = current_sse
            self.centroids = new_centroids
            self._iterations += 1
        self._sse_score = self._sum_of_squared_errors(X, assignments)
        return assignments


In [8]:
unique_labels = np.unique(labels)
no_of_clusters = unique_labels.size
MAX_ITERATIONS = 100

First Running Euclidean K-means

In [9]:
euclidean_kmeans_m = KMeansClustering(k=no_of_clusters)
euclidean_kmeans_m_labels = euclidean_kmeans_m.fit(X=data, max_iterations=MAX_ITERATIONS)

In [10]:
sse_euclidean_m = euclidean_kmeans_m.get_sum_of_squared_error()
sse_euclidean_m

44299172.32931266

Second Running Cosine K-means

In [11]:
cosine_distances = pairwise_distances(data, metric='cosine')
cosine_kmeans_m = KMeansClustering(k=no_of_clusters)
cosine_kmeans_m_labels = cosine_kmeans_m.fit(cosine_distances, max_iterations=MAX_ITERATIONS)

In [12]:
sse_cosine_m = cosine_kmeans_m.get_sum_of_squared_error()
sse_cosine_m

3439.380284784723

Third Running Jarcard K-means

In [13]:
jaccard_distances = pairwise_distances(data, metric='hamming')
jaccard_kmeans_m = KMeansClustering(k=no_of_clusters)
jaccard_kmeans_m_labels = jaccard_kmeans_m.fit(X=jaccard_distances, max_iterations=MAX_ITERATIONS)

In [14]:
sse_jaccard_m = jaccard_kmeans_m.get_sum_of_squared_error()
sse_jaccard_m

1381.9281473531667

In [15]:
print("SSE of Euclidean K-means:", sse_euclidean_m,"\tSSE of Cosine K-means:", sse_cosine_m ,"\tSSE of Jarcard K-means:", sse_jaccard_m)

SSE of Euclidean K-means: 44299172.32931266 	SSE of Cosine K-means: 3439.380284784723 	SSE of Jarcard K-means: 1381.9281473531667


**Q2: Compare the accuracies of Euclidean-K-means Cosine-K-means, Jarcard-K-means. First,** 
**label each cluster using the majority vote label of the data points in that cluster. Later, compute**
**the predictive accuracy of Euclidean-K-means, Cosine-K-means, Jarcard-K-means. Which metric** 
**is better?**

In [18]:
def label_clusters(labels, true_labels):
    unique_labels = np.unique(true_labels)
    cluster_labels = np.zeros(len(labels), dtype=np.int_)
    for cluster in range(no_of_clusters):
        cluster_indices = np.where(labels == cluster)[0]
        cluster_true_labels = true_labels[cluster_indices]
        majority_label = np.argmax([np.sum(cluster_true_labels == label) for label in unique_labels])
        cluster_labels[cluster_indices] = majority_label
    return cluster_labels


cluster_labels_euclidean = label_clusters(euclidean_kmeans_m_labels, labels)
cluster_labels_cosine = label_clusters(cosine_kmeans_m_labels, labels)
cluster_labels_jaccard = label_clusters(jaccard_kmeans_m_labels, labels)

accuracy_euclidean = accuracy_score(labels, cluster_labels_euclidean)
accuracy_cosine = accuracy_score(labels, cluster_labels_cosine)
accuracy_jaccard = accuracy_score(labels, cluster_labels_jaccard)

In [19]:
print("Accuracy of Euclidean K-means: ",accuracy_euclidean)
print("Accuracy of Cosine K-means: ",accuracy_cosine)
print("Accuracy of Jaccard K-means: ",accuracy_jaccard)

Accuracy of Euclidean K-means:  0.5874
Accuracy of Cosine K-means:  0.441
Accuracy of Jaccard K-means:  0.2699


**Q3: Set up the same stop criteria: “when there is no change in centroid position OR when the**
**SSE value increases in the next iteration OR when the maximum preset value (e.g., 500, you**
**can set the preset value by yourself) of iteration is complete”, for Euclidean-K-means, Cosine-Kmeans, Jarcard-K-means. Which method requires more** **iterations and times to converge?**

In [20]:
euclidean_iterations = euclidean_kmeans_m.get_iterations_to_converge()
euclidean_iterations

85

In [21]:
cosine_iterations = cosine_kmeans_m.get_iterations_to_converge()
cosine_iterations

53

In [22]:
jaccard_iterations = jaccard_kmeans_m.get_iterations_to_converge()
jaccard_iterations

4

**Q4: Compare the SSEs of Euclidean-K-means Cosine-K-means, Jarcard-K-means with respect to 
the following three terminating conditions:**

**• when there is no change in centroid position**

**• when the SSE value increases in the next iteration**

**• when the maximum preset value (e.g., 100) of iteration is complete**

1. No change in centroid Position

In [23]:
euclidean_kmeans_no_change = KMeansClustering(k=no_of_clusters, stopping_criterion="no_change")
euclidean_kmeans_no_change_predicted_labels = euclidean_kmeans_no_change.fit(data, max_iterations=MAX_ITERATIONS)
sse_euclidean_kmeans_no_change = euclidean_kmeans_no_change.get_sum_of_squared_error()

In [24]:
sse_euclidean_kmeans_no_change

39845615.50903106

In [25]:
cosine_kmeans_no_change = KMeansClustering(k=no_of_clusters, stopping_criterion="no_change")
cosine_kmeans_no_change_predicted_labels = cosine_kmeans_no_change.fit(cosine_distances, max_iterations=MAX_ITERATIONS)
sse_cosine_kmeans_no_change = cosine_kmeans_no_change.get_sum_of_squared_error()
sse_cosine_kmeans_no_change

5706.89459312199

In [26]:
jaccard_kmeans_no_change = KMeansClustering(k=no_of_clusters, stopping_criterion="no_change")
jaccard_kmeans_no_change_predicted_labels = jaccard_kmeans_no_change.fit(jaccard_distances, max_iterations=MAX_ITERATIONS)
sse_jaccard_kmeans_no_change = jaccard_kmeans_no_change.get_sum_of_squared_error()
sse_jaccard_kmeans_no_change

1287.6987060439546

2. Increase in SSE value

In [27]:
euclidean_kmeans_increase = KMeansClustering(k=no_of_clusters, stopping_criterion="increase_sse")
euclidean_kmeans_increase_predicted_labels = euclidean_kmeans_increase.fit(data, max_iterations=MAX_ITERATIONS)
sse_euclidean_kmeans_increase = euclidean_kmeans_increase.get_sum_of_squared_error()

In [28]:
sse_euclidean_kmeans_increase

126019553.5266952

In [29]:
cosine_kmeans_increase = KMeansClustering(k=no_of_clusters, stopping_criterion="increase_sse")
cosine_kmeans_increase_predicted_labels = cosine_kmeans_increase.fit(cosine_distances, max_iterations=MAX_ITERATIONS)
sse_cosine_kmeans_increase = cosine_kmeans_increase.get_sum_of_squared_error()
sse_cosine_kmeans_increase

10402.98019516619

In [31]:
jaccard_kmeans_increase = KMeansClustering(k=no_of_clusters, stopping_criterion="increase_sse")
jaccard_kmeans_increase_predicted_labels = jaccard_kmeans_increase.fit(jaccard_distances, max_iterations=MAX_ITERATIONS)
sse_jaccard_kmeans_increase = jaccard_kmeans_increase.get_sum_of_squared_error()

In [32]:
sse_jaccard_kmeans_increase

2543.7668069406354

3. Max iterations 

In [34]:
euclidean_kmeans_max = KMeansClustering(k=no_of_clusters, stopping_criterion="max_iterations")
euclidean_kmeans_max_predicted_labels = euclidean_kmeans_max.fit(data, max_iterations=MAX_ITERATIONS)
sse_euclidean_kmeans_max = euclidean_kmeans_max.get_sum_of_squared_error()
sse_euclidean_kmeans_max

42670740.59088298

In [35]:
cosine_kmeans_max = KMeansClustering(k=no_of_clusters, stopping_criterion="max_iterations")
cosine_kmeans_max_predicted_labels = cosine_kmeans_max.fit(cosine_distances, max_iterations=MAX_ITERATIONS)
sse_cosine_kmeans_max = cosine_kmeans_max.get_sum_of_squared_error()
sse_cosine_kmeans_max

9606.027938459452

In [30]:
jaccard_kmeans_max = KMeansClustering(k=no_of_clusters, stopping_criterion="max_iterations")
jaccard_kmeans_max_predicted_labels = jaccard_kmeans_increase.fit(jaccard_distances, max_iterations=MAX_ITERATIONS)
sse_jarcard_kmeans_max = jaccard_kmeans_increase.get_sum_of_squared_error()
sse_jarcard_kmeans_max

1298.5214040520063